<a href="https://colab.research.google.com/github/roushankumar754/JupyterNotebook/blob/main/AnimalBinaryClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the input shape for your images
input_shape = (224, 224, 3)

# Load the pre-trained MobileNet model and remove the top layer
base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')

# Add a new top layer for classification
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Combine the base model and new top layers into a single model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with a binary cross-entropy loss function and an Adam optimizer
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Use data augmentation to generate additional training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Load the training and validation datasets using the ImageDataGenerator
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/classification/training',
                                                    target_size=input_shape[:2],
                                                    batch_size=32,
                                                    class_mode='binary')

validation_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/classification/validating',
                                                         target_size=input_shape[:2],
                                                         batch_size=32,
                                                         class_mode='binary')

# Train the model for a few epochs
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=10,
          validation_data=validation_generator,
          validation_steps=len(validation_generator))

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


9406464/9406464 [==============================] - 0s 0us/step
Found 3568 images belonging to 2 classes.
Found 890 images belonging to 2 classes.
Epoch 1/10
112/112 [==============================] - 1058s 9s/step - loss: 0.2702 - accuracy: 0.8999 - val_loss: 0.1403 - val_accuracy: 0.9539
Epoch 2/10
112/112 [==============================] - 308s 3s/step - loss: 0.1242 - accuracy: 0.9560 - val_loss: 0.1612 - val_accuracy: 0.9416
Epoch 3/10
112/112 [==============================] - 303s 3s/step - loss: 0.1138 - accuracy: 0.9557 - val_loss: 0.1251 - val_accuracy: 0.9596
Epoch 4/10
112/112 [==============================] - 302s 3s/step - loss: 0.0944 - accuracy: 0.9680 - val_loss: 0.1313 - val_accuracy: 0.9562
Epoch 5/10
112/112 [==============================] - 319s 3s/step - loss: 0.0885 - accuracy: 0.9683 - val_loss: 0.1373 - val_accuracy: 0.9483
Epoch 6/10
112/112 [==============================] - 293s 3s/step - loss: 0.0781 - accuracy: 0.9697 - val_loss: 0.1239 - val_accuracy: 0.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="/content/model.tflite")
interpreter.allocate_tensors()

# Get the input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the image
img = Image.open("/content/drive/MyDrive/20160212_064946.jpg")
img = img.resize((224, 224))
img = np.array(img, dtype=np.float32)
img = img / 255.0
img = np.expand_dims(img, axis=0)

# Run inference on the model
interpreter.set_tensor(input_details[0]['index'], img)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
prediction = output_data[0][0]
print(prediction)
# Print the predicted class (0 or 1)
if prediction < 0.5:
    print("The image is not an animal.")
else:
    print("The image is an animal.")


0.0011338702
The image is not an animal.
